# Tech Challenge Fase 3 - Modelo Não-Supervisionado
## Análise Exploratória e Clustering

Este notebook demonstra a aplicação de algoritmos de clustering não-supervisionado nos dados de voos.

**Algoritmos Utilizados:**
- K-Means
- DBSCAN

**Bibliotecas:**
- scikit-learn
- pandas
- numpy
- matplotlib
- seaborn

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

from data_loader import DataLoader
from clustering_models import ClusteringAnalyzer

# Configurações de visualização
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("✓ Bibliotecas importadas com sucesso!")

## 1. Carregamento dos Dados

Vamos carregar uma amostra dos dados de voos para análise.

In [ ]:
# Carregar dados (amostra para análise rápida)
data_loader = DataLoader(base_path="./")
data_loader.load_data(sample_size=100000)

# Mostrar resumo
summary = data_loader.get_summary()
print("\n" + "="*70)
print("RESUMO DOS DADOS")
print("="*70)
for key, value in summary.items():
    print(f"{key}: {value}")

## 2. Processamento e Clustering dos Aeroportos

In [ ]:
# Obter estatísticas dos aeroportos
df_airports = data_loader.get_airport_statistics()

# Selecionar features
feature_columns = [
    'total_flights', 'avg_departure_delay', 'std_departure_delay',
    'avg_arrival_delay', 'std_arrival_delay', 'avg_distance',
    'cancellation_rate', 'diversion_rate', 'avg_air_time',
    'LATITUDE', 'LONGITUDE'
]

# Preparar features
X, df_clean = data_loader.prepare_features_for_clustering(df_airports, feature_columns)
print(f"Shape dos dados: {X.shape}")

In [ ]:
# Buscar k ótimo
analyzer = ClusteringAnalyzer()
df_k_results = analyzer.find_optimal_k(X, k_range=range(2, 11))

best_k = int(df_k_results.loc[df_k_results['silhouette_score'].idxmax(), 'k'])
print(f"\nMelhor k: {best_k}")

In [ ]:
# K-Means
kmeans_result = analyzer.kmeans_clustering(X, n_clusters=best_k)

# DBSCAN
dbscan_result = analyzer.dbscan_clustering(X, eps=0.5, min_samples=5)

# Comparar
df_comparison = analyzer.compare_models()
print(df_comparison)

In [ ]:
# Visualizar clusters em 2D (PCA)
pca = PCA(n_components=2)
X_2d = pca.fit_transform(X)

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# K-Means
scatter1 = axes[0].scatter(X_2d[:, 0], X_2d[:, 1], c=kmeans_result['labels'], 
                           cmap='viridis', alpha=0.6, edgecolors='w', linewidth=0.5)
axes[0].set_title(f'K-Means (k={best_k})')
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})')
plt.colorbar(scatter1, ax=axes[0])

# DBSCAN
scatter2 = axes[1].scatter(X_2d[:, 0], X_2d[:, 1], c=dbscan_result['labels'], 
                           cmap='viridis', alpha=0.6, edgecolors='w', linewidth=0.5)
axes[1].set_title('DBSCAN')
axes[1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})')
axes[1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})')
plt.colorbar(scatter2, ax=axes[1])

plt.tight_layout()
plt.show()

In [ ]:
# Melhor modelo
best_model = analyzer.get_best_model()
df_clean['cluster'] = best_model['labels']

# Top aeroportos por cluster
for cluster_id in sorted(df_clean['cluster'].unique()):
    if cluster_id >= 0:
        print(f"\n=== CLUSTER {cluster_id} ===")
        cluster_data = df_clean[df_clean['cluster'] == cluster_id]
        top = cluster_data.nlargest(5, 'total_flights')[['AIRPORT', 'CITY', 'STATE', 'total_flights']]
        print(top.to_string(index=False))

## Conclusão

Os clusters identificados representam diferentes perfis de aeroportos:

- **Aeroportos Hub vs Regionais**
- **Níveis de Congestionamento**
- **Padrões Geográficos**
- **Eficiência Operacional**